In [ ]:
# run for first time setup
! pip install ipywidgets
! jupyter nbextension enable --py widgetsnbextension

In [33]:
import sys
sys.path.append("..\\")
import login
login.connect()
import datajoint as dj

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from ipywidgets.widgets import Layout, Label

In [42]:
import inspect
out = widgets.Output()

available_schema = dj.list_schemas()
schema_dropdown = widgets.Dropdown(options = available_schema, value = "common_mice")
dj_schema = dj.Schema("common_mouse")
mod = dj.VirtualModule("temp.py", "common_mice", create_schema = False, create_tables=False)

def get_table_names(virtualmodule):
    names = []
    for i in inspect.getmembers(virtualmodule):
        a = getattr(virtualmodule, i[0])
        if type(a) == dj.user_tables.TableMeta:
            names.append(i[0])
    return names

tables = get_table_names(mod)
table_dropdown = widgets.Dropdown(options = tables)

@out.capture(clear_output=True, wait=True)
def schema_update(*args):
    mod = dj.VirtualModule("temp.py", schema_dropdown.value, create_schema = False, create_tables=False)
    tables = get_table_names(mod)
    table_dropdown.options = tables
schema_dropdown.observe(schema_update, "value")

def get_attribute_names(table_class):
    names = list(table_class().heading.attributes.keys())
    return names

@out.capture(clear_output=True, wait=True)
def show_table():
    mod = dj.VirtualModule("temp.py", schema_dropdown.value, create_schema = False, create_tables=False)
    table_class = getattr(mod, table_dropdown.value)
    table_attributes = get_attribute_names(table_class)
    display(table_class())
    return table_class
#table_dropdown.observe(show_table, "value")

def generate_attribute_widgets(table_class):
    w_attr = {}
    attributes = table_class.heading.attributes
    for i in attributes:
        if "varchar" in attributes[i].type:
            w_attr["attr_" + i] = widgets.Text(description = i)
    return w_attr

update_button = widgets.Button(description="Update")
def update_display(*args):
    global vbox
    vbox.close()
    w = {}
    w['schema'] = schema_dropdown
    w['table'] = table_dropdown
    w['button'] = update_button
    #print(controls.children)
    out.clear_output()
    
    table_class = show_table()
    w_attr = generate_attribute_widgets(table_class)
    controls1 = widgets.VBox(list(w.values()))
    label_layout = Layout(width='100px',height='30px')
    controls2 = widgets.VBox([Label("Attributes",layout=label_layout)] + list(w_attr.values()), layout = Layout(border='1px solid black'))
    controls3 = widgets.HBox([controls1, controls2])
    vbox = widgets.VBox([controls3, out])
    display(vbox)
update_button.on_click(update_display)

w = {}
w['schema'] = schema_dropdown
w['table'] = table_dropdown
w['button'] = update_button
controls = widgets.VBox(list(w.values()))
vbox = widgets.VBox([controls, out])
display(vbox)

#interact(show_table, schema = schema_dropdown, table = table_dropdown)